In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install -q langchain sentence-transformers transformers accelerate faiss-cpu pypdf python-docx gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 11.3 MB/s eta 0:00:00


In [3]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [18]:
import os, glob, json
import torch
from pathlib import Path

from langchain.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader, CSVLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

from transformers import pipeline

print("Torch CUDA available:", torch.cuda.is_available())
DEVICE = 0 if torch.cuda.is_available() else -1

Torch CUDA available: True


In [10]:
pip install "unstructured[pdf]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.9/527.9 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.2 MB/s eta 0:00:00


In [14]:
DATA_DIR = "/content/drive/MyDrive/AI projects/PDF"
# DirectoryLoader will pick up many file types automatically
loader = DirectoryLoader(DATA_DIR, glob="**/*")
raw_docs = loader.load()
print("Loaded documents:", len(raw_docs))

# If you want to inspect sources:
for d in raw_docs[:6]:
    print("----source:", d.metadata.get("source", "unknown")[:200])
    print("text sample:", d.page_content[:200].replace("\n", " "))


Loaded documents: 1
----source: /content/drive/MyDrive/AI projects/PDF/NLP with Transformer models.pdf
text sample: Revised Edition Natural Language Processing with Transformers  Building Language Applications with Hugging Face  v  o  T  n  u  &  W  n  s  W  e  t  a  r  o  r  l  a  l  l  f  ,  Lewis Tunstall, Leand


In [16]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=200)
docs = splitter.split_documents(raw_docs)
print("Number of Chunks:", len(docs))

for d in docs[:3]:
  print('meta:', d.metadata)
  print('len text:', len(d.page_content))

Number of Chunks: 802
meta: {'source': '/content/drive/MyDrive/AI projects/PDF/NLP with Transformer models.pdf'}
len text: 1101
meta: {'source': '/content/drive/MyDrive/AI projects/PDF/NLP with Transformer models.pdf'}
len text: 1138
meta: {'source': '/content/drive/MyDrive/AI projects/PDF/NLP with Transformer models.pdf'}
len text: 683


In [20]:
from langchain.embeddings import HuggingFaceEmbeddings
# Use a small, free embedding model
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)

# Build an in-memory FAISS vector store
vectordb = FAISS.from_documents(docs, embeddings)
print("FAISS index created with", len(docs), "vectors")


/tmp/ipython-input-207675083.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FAISS index created with 802 vectors


In [21]:
PERSIST_DIR = "/content/drive/MyDrive/rag_index"
os.makedirs(PERSIST_DIR, exist_ok=True)
vectordb.save_local(PERSIST_DIR)
print("Saved index to", PERSIST_DIR)

# To load later (new session):
# from langchain.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)
# vectordb = FAISS.load_local(PERSIST_DIR, embeddings)


Saved index to /content/drive/MyDrive/rag_index


In [22]:
LLM_MODEL = "google/flan-t5-base"   # small, free, text2text
pipe = pipeline(
    "text2text-generation",
    model=LLM_MODEL,
    device=DEVICE,
    max_new_tokens=256,
    do_sample=False  # deterministic answers
)
llm = HuggingFacePipeline(pipeline=pipe)

# Build retrieval chain; ask it to return source_documents so we can show citations
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # simple approach: stuff retrieved docs into prompt
    retriever=vectordb.as_retriever(search_kwargs={"k": 4}),
    return_source_documents=True
)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-1617222947.py:9: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [42]:
def ask(query):
    res = qa({"query": query})
    answer = res["result"]
    srcs = res.get("source_documents", [])
    print("→\nANSWER:\n", answer)
    print("\nSOURCES:")
    for s in srcs:
        print("-", s.metadata.get("source", "unknown"))

# Example:
ask("how many freely available models The Hugging Face Hub hosts ")


→
ANSWER:
 over 20,000 freely available models

SOURCES:
- /content/drive/MyDrive/AI projects/PDF/NLP with Transformer models.pdf
- /content/drive/MyDrive/AI projects/PDF/NLP with Transformer models.pdf
- /content/drive/MyDrive/AI projects/PDF/NLP with Transformer models.pdf
- /content/drive/MyDrive/AI projects/PDF/NLP with Transformer models.pdf


In [43]:
import gradio as gr

def respond(query):
    res = qa({"query": query})
    answer = res["result"]
    srcs = [d.metadata.get("source", "unknown") for d in res.get("source_documents", [])]
    return answer, "\n".join(srcs)

demo = gr.Interface(
    fn=respond,
    inputs=gr.Textbox(lines=2, placeholder="Ask about your documents..."),
    outputs=[gr.Textbox(label="Answer"), gr.Textbox(label="Sources")],
    title="CHATBOT for NoteBooks"
)
demo.launch(share=True)  # share=True gives a public temporary link


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://201771f0606de718ad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
